In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## Basic RAG with Weaviate

Now - let's try performing RAG with the chunks that we've created. 

We will:
- Load & chunk a document
- Add the chunks to Weaviate, and generate vectors
- And perform RAG

We assume some familiarity with Weaviate here. 

(If not, check out the [Weaviate Quickstart](https://docs.weaviate.io/weaviate/quickstart), or ask questions in the live session!)

### Load and chunk a document

In [2]:
from pathlib import Path

def get_chunks_using_markers(src_text: str) -> list[str]:
    """
    Split the source text into chunks using markers.
    """
    marker = "\n##"

    # Split by marker and reconstruct with markers (except first chunk)
    parts = src_text.split(marker)
    chunks = []

    # Add first chunk if it exists and isn't empty
    if parts[0].strip():
        chunks.append(parts[0].strip())

    # Add remaining chunks with markers reattached
    for part in parts[1:]:
        if part.strip():
            chunks.append(marker + part.strip())

    return chunks


md_file = Path("data/parsed/hai_ai_index_report_2025_chapter_2-parsed-text.md")
md_text = md_file.read_text(encoding="utf-8")
chunk_texts = get_chunks_using_markers(md_text)

### Set up Weaviate

Connection code looks like this:

```python
client = weaviate.connect_to_embedded(
    version="1.32.5",
    headers={
        "X-Anthropic-Api-Key": os.getenv("ANTHROPIC_API_KEY"),
        "X-Cohere-Api-Key": os.getenv("COHERE_API_KEY")
    },
)
```

In [3]:

import utils

# Helper function to connect to Weaviate
client = utils.connect_to_weaviate()

### Set up a collection

In [4]:
client.collections.delete("Chunks")

In [5]:
from weaviate.classes.config import Property, DataType, Configure, Tokenization

client.collections.create(
    name="Chunks",
    properties=[
        Property(
            name="document_title",
            data_type=DataType.TEXT,
        ),
        Property(
            name="chunk",
            data_type=DataType.TEXT,
        ),
        Property(
            name="chunk_number",
            data_type=DataType.INT,
        ),
        Property(
            name="filename",
            data_type=DataType.TEXT,
            tokenization=Tokenization.FIELD
        ),
    ],
    vector_config=[
        # Add `Configure.Vectors.text2vec_cohere` vector to the collection with:
        # name: "default", source properties: ["document_title", "chunk"], model: "embed-v4.0"
        # BEGIN_SOLUTION
        Configure.Vectors.text2vec_cohere(
            name="default",
            source_properties=["document_title", "chunk"],
            model="embed-v4.0"
        )
        # END_SOLUTION
    ],
    generative_config=Configure.Generative.anthropic(
        model="claude-3-5-haiku-latest"
    )
)

### Import data

In [6]:
chunks = client.collections.get("Chunks")

In [7]:
from tqdm import tqdm

with chunks.batch.fixed_size(batch_size=100) as batch:
    for i, chunk_text in tqdm(enumerate(chunk_texts)):
        obj = {
            "document_title": "Stanford HAI Report 2025",
            "filename": "data/pdfs/hai_ai_index_report_2025_chapter_2.pdf",
            "chunk": chunk_text,
            "chunk_number": i + 1,
        }

        # Add object to batch for import with (batch.add_object())
        # BEGIN_SOLUTION
        batch.add_object(
            properties=obj
        )
        # END_SOLUTION

266it [00:00, 15871.24it/s]


### RAG queries



In [ ]:
# Try a RAG query with:
# query (what to search for): "Agentic AI use cases" and
# grouped_task (prompt): "Briefly, what use cases and benefits of agentic AI are mentioned here?"
# limit (how many objects to fetch): 10
# BEGIN_SOLUTION
response = chunks.generate.hybrid(
    query="Agentic AI use cases",
    limit=5,
    grouped_task="Briefly, what use cases and benefits of agentic AI are mentioned here?"
)
# END_SOLUTION

print("Query response:")
print(response.generative.text)

Query response:
Based on the provided text, here are the key use cases and benefits of agentic AI mentioned:

Use Cases:
1. Academic research assistance
2. Scheduling meetings
3. Online shopping facilitation
4. Vacation booking
5. Operating in household and gaming environments
6. Interacting with mobile and web applications
7. Visual design tasks (like CSS debugging)

Benefits:
1. Ability to operate autonomously or semi-autonomously in specific environments
2. Capability to accomplish complex goals across diverse and dynamic contexts
3. Increasing multimodality, allowing interaction across different types of environments
4. Potential to extend AI's functionality beyond traditional linguistic interactions

The text also notes that while agentic AI is promising, current models are still far from being ready for direct deployment, with top-performing models achieving only around 36.2% success rate in benchmarks.


### Recap - what's happening under the hood

![assets/llm_3_rag_weaviate.png](assets/llm_3_rag_weaviate.png)

We can review the passages:

In [9]:
print("Supporting passages:")
for o in response.objects:
    print(f"\n> Object: {o.uuid}:")
    print(o.properties['chunk'][:200]+"...")

Supporting passages:

> Object: bed7cd7c-bef4-4000-8187-2b37f2b7aa8f:

##Gemini 2.0 in an agentic work fl ow

Source: AI Index, 2025

<!-- image -->...

> Object: 5bba3d76-5623-458c-9388-5ff6329a5e78:

##2.8 AI Agents Chapter 2: Technical Performance

AI agents, autonomous or semiautonomous systems designed to operate within  speci fi c  environments to accomplish goals, represent an  exciting  fro...

> Object: 8b9503cd-c76c-47e2-9cdc-b8d7883876a9:

##2.8 AI Agents

For decades, the topic of AI agents has been widely discussed in the AI community, yet few benchmarks have achieved widespread adoption, including those featured in last year's Index...

> Object: 261525a0-224d-4782-a23b-6736ad2bf673:

##2.8 AI Agents Chapter 2: Technical Performance

<!-- image -->

VAB presents a signi fi cant challenge for AI systems. The topperforming model, GPT-4o, achieves an overall success rate of just 36.2...

> Object: 1a3329ae-89af-468a-870b-147be4ec84a4:

##VisualAgentBench

VisualAgentBench  (

In [10]:
client.close()

{"build_git_commit":"08d409a988","build_go_version":"go1.25.0","build_image_tag":"HEAD","build_wv_version":"1.32.5","error":"cannot find peer","level":"error","msg":"transferring leadership","time":"2025-09-16T12:03:11+01:00"}
